# Creation of a database SQlite

In this kernel, we will show how create a fonctional database SQlite. This database makes it possible not to ask the queries every time.

In [1]:
# Import libraries usefull

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup

from collections import Counter
from operator import itemgetter
import pandas as pd

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [2]:
import sparql_functions as spqf # It's made-home library created by Francesco Beretta
# so they must to be in the same folder as this file.

In [3]:
### Connexion to a SQL Database or create it if it doesn't exist yet
# into the indicated folder

cn = sql.connect('data/sparql_queries.db')

In [4]:
## A  cursor is an object that allows you to execute queries on the database by isolating them properly
c = cn.cursor()

##  https://www.sqlite.org/lang_datefunc.html
# this checks then it is well connect to the database with the current local time
c.execute("SELECT datetime('now', 'localtime')")
print(c.fetchone())

('2021-04-22 11:14:53',)


In [5]:
## On ferme la base de données
cn.close()

In [10]:
db = 'data/sparql_queries.db'   ### 'data/database.db'

In [11]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# c.execute('''DROP TABLE query;''')

c.execute('''CREATE TABLE query (pk_query INTEGER PRIMARY KEY,
label VARCHAR (100),
description TEXT,
project VARCHAR (150),
sparql_endpoint VARCHAR (250),
query TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# disable # cn.commit()
cn.close()

In [12]:
cn = sql.connect(db)
c = cn.cursor()

# Create table
# c.execute('''DROP TABLE result;''')

c.execute('''CREATE TABLE result (pk_result INTEGER PRIMARY KEY,
fk_query INTEGER REFERENCES "query" (pk_query) MATCH SIMPLE,
description TEXT,
result TEXT,
insert_data_into TEXT,
notes TEXT, 
timestmp text DEFAULT ((datetime('now','localtime'))))
''')

# Save (commit) the changes
# disable # cn.commit()
cn.close()

In [18]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'entity'
# STOP #  
#c.execute('''DROP TABLE entity''')
#"""
c.execute('''CREATE TABLE entity
             (pk_entity INTEGER PRIMARY KEY, local_authority BOOLEAN DEFAULT TRUE, 
             fk_same_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, entity_class TEXT, source TEXT,
             fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             entity_std_name TEXT, notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, entity_class, source))''')
#"""

# Save (commit) the changes 
# disable # cn.commit()

cn.close()

OperationalError: table entity already exists

In [67]:
cn = sql.connect(db)
c = cn.cursor()

# Create table 'property'
# STOP # c.execute('''DROP TABLE property''')
c.execute('''CREATE TABLE property
             (pk_property INTEGER PRIMARY KEY, 
             fk_entity INTEGER REFERENCES [entity] (pk_entity) MATCH SIMPLE,
             uri_entity TEXT, property TEXT, value TEXT, 
             source TEXT, quality INTEGER, fk_query_as_source INTEGER REFERENCES [query] (pk_query) MATCH SIMPLE,
             additional_property_1 TEXT, value_1 TEXT,
             additional_property_2 TEXT, value_2 TEXT,
             additional_property_3 TEXT, value_3 TEXT,
             notes TEXT, 
             timestmp text DEFAULT ((datetime('now','localtime'))),
             UNIQUE (uri_entity, property, value, source))''')


# Save (commit) the changes 
# disable #  cn.commit()

cn.close()

In [15]:
### Fonction qui récupère et met en forme le temps au moment de l'exécution

# définition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# exécution
timestamp_formatted_for_file_name()

'20210422_095219'

In [25]:
### Définir les adresses des fichiers, l'existant et celui à créer
original_db = 'data/sparql_queries.db'

##  Noter que la différence de suffixe, en soi totalement arbitraire, 
#  dépend du fait que dans la configuration de .gitignore, .sqlite est exclu du verisonnement GIT
#  contrairement à .db qui est versionné
timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [26]:
### Créer une copie de sauvegarde avec timestamp du fichier (versionnement manuel)
# ATTENTION : la base de données doit exister à l'endroit indiqué !
## Cette requête n'est utile que si des modifications en écriture vont être apportées à la base de données,
# afin de préserver le dernier état avant modification

## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210422_093251.sqlite'